<a href="https://colab.research.google.com/github/Pbrillan/CEIA/blob/main/VPC2/TP1_CV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def zero_padding( X, pad):
  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))
  return X_padded
def convolve( X, W, b):
  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)
  return Z

In [ ]:
class conv2dim:
  def __init__(self):
    self.model = 1

  def conv_forward(self,layer_input, W, b, stride, padding):
    # Obtengo las dimensiones de la entrada
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

    # Obtengo las dimensiones de los filtros
    (n_C, n_C_prev, filter_size, filter_size) = W.shape

    # Obtengo el Padding
    if (padding == 'SAME'):
      padding = int(((stride-1) * n_H_prev - stride + n_C)/2)
    elif (padding == 'VALID'):
      padding = 0

    # Calculo las dimensiones del volumen de salida de la capa actual
    n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
    n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

    # Inicializo el volumen de salida con ceros
    Z = np.zeros([batch_size, n_C, n_H, n_W])

    # Agrego padding con ceros al volumen de entrada
    layer_input_padded = zero_padding(layer_input, padding)

    # Comienzo iterando sobre cada ejemplo del batch
    for i in range(batch_size):

      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = stride * h
        y_end = y_start + filter_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = stride * w
          x_end = x_start + filter_size

          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
          
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):

            # Obtengo el valor del filtro y bias del canal correspondiente
            filter = W[c, :, :, :]
            bias = b[c]

            # Computo la operación de convolución para esta ventana
            Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
    
    return Z

  def conv_backward(self, dLdO, layer_input, W, b, stride, padding):

    # Obtengo las dimensiones de la entrada
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

    # Obtengo las dimensiones de los filtros
    (n_C, n_C_prev, filter_size, filter_size) = W.shape

    # Obtengo el Padding 
    if (padding == 'SAME'):
      padding = int(((stride-1) * n_H_prev - stride + n_C)/2)
    elif (padding == 'VALID'):
      padding = 0
    
    # Calculo las dimensiones del volumen de salida de la capa actual
    n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
    n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

    # Inicializo el volumen de salida con ceros
    dLdF = np.zeros([batch_size, n_C, n_H, n_W])
    dLdX = np.zeros([batch_size, n_C, n_H, n_W])


    # Agrego padding con ceros al volumen de entrada
    layer_input_padded = zero_padding(layer_input, padding)

    # Comienzo iterando sobre cada ejemplo del batch
    for i in range(batch_size):

      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = stride * h
        y_end = y_start + filter_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = stride * w
          x_end = x_start + filter_size

          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
          
          # Itero sobre la cantidad de canales 
          for c in range(n_C):
            # Obtengo el valor del filtro y bias del canal correspondiente
            bias = b[c]
            filter = np.flip(W[c, :, :, :])

            # Computo la operación de convolución para esta ventana
            dLdX[i, c, :, :] = convolve(dLdO, filter, bias)
            dLdF[i, c, h, w] = convolve(slice_from_input_padded, dLdO, bias)
    
    return dLdX, dLdF

In [ ]:
# Dimensiones entrada
batch_size = 1
input_height, input_width = (7, 7)
input_channels = 3

# Dimensiones convolucional
filters = 3
filter_size = 3
stride = 1
padding = "SAME"

# Variables prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)
modelo2 = conv2dim()
Z = modelo2.conv_forward(test_array,W,b,stride,padding)
dz = np.random.randn(input_channels, filter_size, filter_size)
d1,d2 = modelo2.conv_backward(dz, test_array,W,b,stride,padding)

In [ ]:
import torch

In [ ]:
  class ConvModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=input_channels, out_channels=filters, kernel_size=filter_size, stride=stride, padding='same')
    def forward(self, x):
        x = self.conv1(x)
        return x

    def backward(self, grad_output,x):
        fctDataIn = x
        fctWeight = self.wehigt
        tmpDataVar = torch.autograd.Variable(fctDataIn, requires_grad=True)
        tmpWeightParam = torch.nn.Parameter(fctWeight)
        tmpLin = torch.nn.Linear(3, 2, bias=False)
        tmpLin.weight = tmpWeightParam
        tmpLin.zero_grad()
        print(tmpDataVar.data)
        print(tmpWeightParam.data)
        print(grad_output)
        print("still here...")
        tmpLin(tmpDataVar).backward(grad_output)
        print("cannot reach this :( ")
        grad_fctDataIn = tmpDataVar.grad.data
        grad_fctWeight = tmpWeightParam.grad.data
        print(grad_fctDataIn)
        print(grad_fctWeight)
        return grad_fctDataIn, grad_fctWeight

conv_model = ConvModel()

In [ ]:
test_array_tens = torch.tensor(test_array)
test_array_tens.double()
conv_model.double()
Z1  = conv_model.forward(test_array_tens)
#b11, b21 = conv_model.backward(dz,test_array_tens)

In [ ]:
print(Z)


[[[[ -2.37272439   8.70074869  -7.18105581   5.00889894  -1.04964233
      4.70563319   1.29886583]
   [ -3.798321     2.52105151  -3.74691909  -5.96784089   6.02131839
      4.48862425   4.8954502 ]
   [  5.00200291   3.18249855  -3.54149561   5.28815761   4.9268814
      5.20344825   5.8034309 ]
   [  2.50119273  -2.66621929   1.50004096  -4.40481949   1.99248
     -6.39947818   2.57671241]
   [ -5.48584919   3.80173102   1.50355873  -7.02590746  -4.53332348
      7.49533822   0.41664838]
   [ -0.30983235   6.38702203   0.17933682   3.26426505   3.97767399
     -0.51790013  -0.34364156]
   [  3.71631108  -3.79373065   2.5906836   -3.07530253   0.83906988
     -0.31053662  -3.97010228]]

  [[  1.48513016  -3.95285242   2.55785024  -5.06374242   5.54150008
      7.59619261   5.09861065]
   [  0.1827405   12.96660826  -0.66673958   0.89610376  -1.06026548
     12.22967283   6.10859329]
   [  1.52808173  -8.09462511  -1.57938923 -10.60319861  -9.16921094
     -3.98773579   0.31175636]
  

In [ ]:
print(Z1)

tensor([[[[ -2.3727,   8.7007,  -7.1811,   5.0089,  -1.0496,   4.7056,   1.2989],
          [ -3.7983,   2.5211,  -3.7469,  -5.9678,   6.0213,   4.4886,   4.8955],
          [  5.0020,   3.1825,  -3.5415,   5.2882,   4.9269,   5.2034,   5.8034],
          [  2.5012,  -2.6662,   1.5000,  -4.4048,   1.9925,  -6.3995,   2.5767],
          [ -5.4858,   3.8017,   1.5036,  -7.0259,  -4.5333,   7.4953,   0.4166],
          [ -0.3098,   6.3870,   0.1793,   3.2643,   3.9777,  -0.5179,  -0.3436],
          [  3.7163,  -3.7937,   2.5907,  -3.0753,   0.8391,  -0.3105,  -3.9701]],

         [[  1.4851,  -3.9529,   2.5579,  -5.0637,   5.5415,   7.5962,   5.0986],
          [  0.1827,  12.9666,  -0.6667,   0.8961,  -1.0603,  12.2297,   6.1086],
          [  1.5281,  -8.0946,  -1.5794, -10.6032,  -9.1692,  -3.9877,   0.3118],
          [  4.6042,  -1.4462,  -3.8527,  12.5016,  -3.4642,   8.5182,   7.9153],
          [  5.1862,   3.0164,   9.8879,  -3.3876,  -3.4480,  -5.5469,   3.5991],
          [  0

In [1]:

#Z1 = modelo3.forward(torch.tensor(test_array))
#d11,d21 = modelo2.backward(dz,test_array)